In [36]:
import os
import re
import pandas as pd
import numpy as np
import duckdb
import unicodedata
from tqdm import tqdm
from textblob import TextBlob
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from scipy.stats import trim_mean

In [ ]:
def preprocess_runtime(df, k=0.1):
    """
    Preprocesses the runtime information, including handling extreme values
    (movies with runtime > 1000 minutes, possibly in seconds or hours).

    Arguments:
    - df: DataFrame containing the movie data.
    - k: Proportion of values to trim from each end when computing the trimmed mean.

    Returns:
    - DataFrame with cleaned runtime information.
    """

    # Handle movies that are possibly in seconds or hours
    df["runtimeMinutes"] = df["runtimeMinutes"].apply(lambda x: x / 60 if x > 1000 else x)  # Convert seconds to minutes
    df["runtimeMinutes"] = df["runtimeMinutes"].apply(lambda x: x * 60 if x < 5 else x)  # Convert minutes to hours if under 5 mins

    # Compute k-trimmed mean for runtimeMinutes (ignoring NaN values)
    trimmed_mean_runtime = trim_mean(df["runtimeMinutes"].dropna(), proportiontocut=k)

    print(trimmed_mean_runtime)

    # Fill missing values with trimmed mean
    df["runtimeMinutes"].fillna(trimmed_mean_runtime)

    return df

In [37]:
def preprocess_imdb_data(data_path, directors_path, writers_path):
    """
    General preprocessing pipeline for IMDB data.
    
    Arguments:
    - data_path: Path to the train/test/validation data CSV file.
    - directors_path: Path to the directing.json file.
    - writers_path: Path to the writing.json file.
    
    Returns:
    - Cleaned Pandas DataFrame ready for model training or prediction.
    """
    
    # Step 1: Load main dataset
    df = pd.read_csv(data_path)

    # Step 2: Load JSON files (Directors & Writers)
    df_directors = pd.read_json(directors_path)
    df_writers = pd.read_json(writers_path)

    # Step 3: Rename columns for consistency
    df_directors.rename(columns={"movie": "tconst", "director": "director_id"}, inplace=True)
    df_writers.rename(columns={"movie": "tconst", "writer": "writer_id"}, inplace=True)

    # Step 4: Convert nested JSON fields into strings
    df_directors["director_id"] = df_directors["director_id"].astype(str)
    df_writers["writer_id"] = df_writers["writer_id"].astype(str)

    # Step 5: Merge main dataset with Directors & Writers using DuckDB
    con = duckdb.connect()
    con.register("movies", df)
    con.register("directors", df_directors)
    con.register("writers", df_writers)

    query = """
    SELECT 
        movies.*, 
        directors.director_id, 
        writers.writer_id
    FROM movies
    LEFT JOIN directors ON movies.tconst = directors.tconst
    LEFT JOIN writers ON movies.tconst = writers.tconst
    """

    df = con.execute(query).fetchdf()
    con.close()

    # Step 6: Create column year from startYear and endYear
    df['startYear'] = df['startYear'].replace('\\N', np.nan).astype(float)
    df['endYear'] = df['endYear'].replace('\\N', np.nan).astype(float)
    df['Year'] = df['startYear'].fillna(df['endYear'])

    # Step 7: Clean title names
    def normalize_text(text):
        if pd.isna(text):  # Handle missing values
            return ""
        text = str(text)
        text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')  # Remove accents
        text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
        return text.strip()

    def clean_titles(row):
        primary = row['primaryTitle'] if pd.notna(row['primaryTitle']) else ''
        original = row['originalTitle'] if pd.notna(row['originalTitle']) else ''

        if not primary:
            primary = original

        cleaned_title = normalize_text(primary)

        return cleaned_title if cleaned_title else "Unknown Title"

    df['primaryTitle'] = df.apply(clean_titles, axis=1)
    df.rename(columns={'primaryTitle': 'movieTitle'}, inplace=True)

    # Step 8: Compute Title Uniqueness Score
    title_counts = df["movieTitle"].value_counts()
    df["title_uniqueness"] = df["movieTitle"].apply(lambda x: 1 / title_counts[x] if title_counts[x] > 1 else 1)

    # Step 9: Compute Sentiment Score
    df["sentiment_score"] = df["movieTitle"].astype(str).apply(lambda x: TextBlob(x).sentiment.polarity)

    # Step 10: Count words in each title
    df["word_count"] = df["movieTitle"].apply(lambda x: len(x.split()))

    #Step 11 title lenght
    df["title_word_length_std"] = df["movieTitle"].apply(lambda x: np.std([len(word) for word in x.split()]) if len(x.split()) > 1 else 0)
    
    # Step 12: Drop unnecessary columns
    columns_to_drop = ["originalTitle", "endYear", "startYear", "Unnamed: 0"]
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    # Step 13: Handle missing values
    df = preprocess_runtime(df, 0.1)

    # Step 14: Fill missing values for numVotes
    trimmed_mean_votes = trim_mean(df["numVotes"].dropna(), proportiontocut=0.1)
    df["numVotes"] = df["numVotes"].fillna(trimmed_mean_votes)

    # Step 15: Fill missing values for director_id and writer_id
    df["director_id"] = df["director_id"].fillna("unknown")
    df["writer_id"] = df["writer_id"].fillna("unknown")

    # Step 16: Ensure correct data types
    df["Year"] = df["Year"].astype(int)
    df["runtimeMinutes"] = df["runtimeMinutes"].astype(int)
    df["numVotes"] = df["numVotes"].astype(int)

    # Step 17: Ensure each `tconst` is unique
    df = df.groupby("tconst").first().reset_index()
    
    return df


In [38]:
# Define file paths
# Define the base directory
base_data_dir = os.path.join(os.getcwd(), "imdb")

# Generate the list of train file paths
train_files = [os.path.join(base_data_dir, f) for f in os.listdir(base_data_dir) if f.startswith("train-") and f.endswith(".csv")]

# Define paths for directors and writers files
directors_path = os.path.join(base_data_dir, "directing.json")
writers_path = os.path.join(base_data_dir, "writing.json")

# Load JSON files (Directors & Writers)
df_directors = pd.read_json(directors_path)
df_writers = pd.read_json(writers_path)

# Preprocess and merge all training data
df_train = pd.concat([preprocess_imdb_data(file, directors_path, writers_path) for file in train_files], ignore_index=True)

# Preprocess validation and test data
df_val = preprocess_imdb_data(os.path.join(base_data_dir, "validation_hidden.csv"), directors_path, writers_path)
df_test = preprocess_imdb_data(os.path.join(base_data_dir, "test_hidden.csv"), directors_path, writers_path)

# Save cleaned datasets
df_train.to_csv("cleaned/final_training_data_titlefeatures.csv", index=False)
df_val.to_csv("cleaned/final_validation_data_titlefeatures.csv", index=False)
df_test.to_csv("cleaned/final_test_data_titlefeatures.csv", index=False)

print("\n✅ All datasets have been preprocessed and saved!")


✅ All datasets have been preprocessed and saved!


In [ ]:
def calculate_unique_ratio(df, columns=None):
    """
    Calculate the ratio of unique rows to total rows in the DataFrame.

    Arguments:
    - df: DataFrame to analyze
    - columns: List of columns to consider (if None, uses all columns)

    Returns:
    - Dictionary containing unique ratio metrics
    """
    if columns is None:
        columns = df.columns

    total_rows = len(df)
    unique_rows = len(df[columns].drop_duplicates())
    ratio = unique_rows / total_rows

    metrics = {
        "total_rows": total_rows,
        "unique_rows": unique_rows,
        "unique_ratio": ratio
    }

    return metrics

# Add this after your data preprocessing
print("\n🔍 Analyzing unique row ratios...")

# Calculate ratios for all datasets
train_metrics = calculate_unique_ratio(df_train)
val_metrics = calculate_unique_ratio(df_val)
test_metrics = calculate_unique_ratio(df_test)

# Print results
print("\nUnique Row Analysis:")
print(f"Training Data:")
print(f"  - Total Rows: {train_metrics['total_rows']:,}")
print(f"  - Unique Rows: {train_metrics['unique_rows']:,}")
print(f"  - Unique Ratio: {train_metrics['unique_ratio']:.2%}")

print(f"\nValidation Data:")
print(f"  - Total Rows: {val_metrics['total_rows']:,}")
print(f"  - Unique Rows: {val_metrics['unique_rows']:,}")
print(f"  - Unique Ratio: {val_metrics['unique_ratio']:.2%}")

print(f"\nTest Data:")
print(f"  - Total Rows: {test_metrics['total_rows']:,}")
print(f"  - Unique Rows: {test_metrics['unique_rows']:,}")
print(f"  - Unique Ratio: {test_metrics['unique_ratio']:.2%}")

In [ ]:
def handle_duplicates(df, groupby_cols=None, agg_strategy=None):
    """
    Handle duplicate rows using specified aggregation strategies.

    Arguments:
    - df: DataFrame to process
    - groupby_cols: List of columns to identify duplicates (default: all columns except label)
    - agg_strategy: Dictionary of column names and aggregation functions

    Returns:
    - DataFrame with handled duplicates
    """
    if groupby_cols is None:
        groupby_cols = [col for col in df.columns if col != 'label']

    if agg_strategy is None:
        agg_strategy = {
            'runtimeMinutes': 'mean',
            'numVotes': 'sum',
            'startYear': 'first',
            'director_id': 'first',
            'writer_id': 'first',
            'label': 'mode'
        }

    # Count occurrences before deduplication
    total_rows = len(df)
    unique_rows = len(df[groupby_cols].drop_duplicates())

    if total_rows == unique_rows:
        print("No duplicates found!")
        return df

    print(f"\nFound {total_rows - unique_rows:,} duplicate rows")
    print(f"Unique ratio before: {(unique_rows/total_rows):.2%}")

    # Handle duplicates using aggregation
    df_cleaned = df.groupby(groupby_cols, as_index=False).agg(agg_strategy)

    # Print results
    print(f"Rows after duplicate handling: {len(df_cleaned):,}")
    print(f"Unique ratio after: {(len(df_cleaned)/total_rows):.2%}")

    return df_cleaned

# Handle duplicates
print("\n🔍 Handling duplicates...")
df_train = handle_duplicates(df_train)
df_val = handle_duplicates(df_val)
df_test = handle_duplicates(df_test)

In [5]:
# # Define Features & Target
# features = ["Year", "runtimeMinutes", "numVotes", "director_id", "writer_id", "word_count"]
# X_train = df_train[features]
# y_train = df_train["label"]
# X_val = df_val[features]
# X_test = df_test[features]

# # Preprocessing Pipeline
# numeric_features = ["Year", "runtimeMinutes", "numVotes", "word_count"]
# categorical_features = ["director_id", "writer_id"]

# preprocessor = ColumnTransformer(
#     transformers=[
#         ("num", StandardScaler(), numeric_features),
#         ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
#     ]
# )

# # Train Logistic Regression Model
# model = Pipeline([
#     ("preprocessing", preprocessor),
#     ("classifier", SVC(kernel="linear", probability=True))
# ])

# print("🔹 Training model on full training data...")
# model.fit(X_train, y_train)

# # Generate Predictions
# y_val_pred = model.predict(X_val)
# y_test_pred = model.predict(X_test)

# # Save predictions in required format (no headers, single column)
# pd.DataFrame(y_val_pred).to_csv("submissions/validation_predictions_SVM.csv", index=False, header=False)
# pd.DataFrame(y_test_pred).to_csv("submissions/test_predictions_SVM.csv", index=False, header=False)

# print("✅ Predictions saved for submission!")

🔹 Training model on full training data...
✅ Predictions saved for submission!


In [41]:
# Load preprocessed training dataset
df_train = pd.read_csv("cleaned/final_training_data_titlefeatures.csv")

# Apply frequency encoding to categorical variables
for col in ["director_id", "writer_id"]:
    freq_encoding = df_train[col].value_counts(normalize=True)
    df_train[col] = df_train[col].map(freq_encoding)

# Define Features & Target
features = ["Year", "runtimeMinutes", "numVotes", "director_id", "writer_id", "word_count", "title_uniqueness", "title_word_length_std", "sentiment_score"]
X = df_train[features]
y = df_train["label"]  # Only train data has labels

# **NEW: Split training data into train (80%) and validation (20%)**
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing Pipeline (same for all models)
numeric_features = ["Year", "runtimeMinutes", "numVotes", "word_count", "title_uniqueness", "title_word_length_std", "sentiment_score"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
    ]
)
# Create pipeline with preprocessing
pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", SVC(kernel="linear", probability=True))
])

# Train model
pipeline.fit(X_train, y_train)

# Make predictions
y_train_pred = pipeline.predict(X_train)
y_val_pred = pipeline.predict(X_val)

# Evaluate model
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"✅ Training Accuracy: {train_accuracy:.4f}")
print(f"✅ Validation Accuracy: {val_accuracy:.4f}")
print(f"📊 Classification Report for:\n", classification_report(y_val, y_val_pred))


✅ Training Accuracy: 0.7105
✅ Validation Accuracy: 0.7161
📊 Classification Report for:
               precision    recall  f1-score   support

       False       0.70      0.77      0.74       820
        True       0.73      0.66      0.69       772

    accuracy                           0.72      1592
   macro avg       0.72      0.71      0.71      1592
weighted avg       0.72      0.72      0.71      1592

